<a href="https://colab.research.google.com/github/TatsukiIshijima/Colab_Notebooks/blob/main/TensorFlowTTS_tacotron2_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TensorFlowTTS の tacotron2 を TFLite 形式もモデルに変換する

TensorFlowTTS リポジトリの説明や notebooks を参考に tacotron2 の TFLite 形式のモデルに変換をやってみる

TensorFlowTTS をソースからインストールするためにリポジトリをクローン

In [ ]:
!git clone https://github.com/TensorSpeech/TensorFlowTTS.git

Cloning into 'TensorFlowTTS'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 10230 (delta 5), reused 8 (delta 1), pack-reused 10207
Receiving objects: 100% (10230/10230), 110.33 MiB | 35.11 MiB/s, done.
Resolving deltas: 100% (4903/4903), done.


ディレクトリの移動

In [ ]:
%cd TensorFlowTTS

/content/TensorFlowTTS
/content/TensorFlowTTS


TensorFlowTTS をインストール

In [ ]:
pip install .

Processing /content/TensorFlowTTS
     |████████████████████████████████| 320.4MB 52kB/s 
     |████████████████████████████████| 706kB 48.8MB/s 
     |████████████████████████████████| 245kB 61.2MB/s 
     |████████████████████████████████| 225kB 57.9MB/s 
     |████████████████████████████████| 3.5MB 46.4MB/s 
     |████████████████████████████████| 1.3MB 48.7MB/s 
     |████████████████████████████████| 1.7MB 49.7MB/s 
     |████████████████████████████████| 3.1MB 52.5MB/s 
  Cloning https://github.com/repodiac/german_transliterate.git to /tmp/pip-install-mikpm9o1/german-transliterate
  Running command git clone -q https://github.com/repodiac/german_transliterate.git /tmp/pip-install-mikpm9o1/german-transliterate
     |████████████████████████████████| 460kB 54.0MB/s 
     |████████████████████████████████| 20.1MB 1.3MB/s 
     |████████████████████████████████| 20.2MB 173kB/s 
     |████████████████████████████████| 184kB 61.9MB/s 
     |████████████████████████████████| 102kB 15.0

In [ ]:
!pip install tf-nightly

     |████████████████████████████████| 403.7MB 44kB/s 
     |████████████████████████████████| 471kB 60.6MB/s 
     |████████████████████████████████| 14.8MB 224kB/s 
     |████████████████████████████████| 5.9MB 47.2MB/s 
     |████████████████████████████████| 4.0MB 49.3MB/s 
     |████████████████████████████████| 4.0MB 53.9MB/s 
ERROR: umap-learn 0.5.0 has requirement numba>=0.49, but you'll have numba 0.48.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.34.1 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 2.3.0 which is incompatible.
ERROR: tensorflow-gpu 2.3.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompat

MelGAN や Tacotron2 の学習モデルをダウンロードし自分の Google Drive へアップロードした。そして自分の Google Drive をマウントし、学習モデルをそこから読み込めるようにした

In [9]:
from google.colab import drive
# Google Drive のマウント
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import numpy as np
import soundfile as sf
import yaml
import tensorflow as tf

from tensorflow_tts.processor import LJSpeechProcessor
# ここでエラー（最新のTensorflow-TTSにはsymbols と symbol_to_id は定義されていないため？）
# from tensorflow_tts.processor.ljspeech import symbols, _symbol_to_id

from tensorflow_tts.configs import Tacotron2Config
from tensorflow_tts.configs import MelGANGeneratorConfig

from tensorflow_tts.models import TFTacotron2
from tensorflow_tts.models import TFMelGANGenerator

from IPython.display import Audio
print(tf.__version__)

2.5.0-dev20210204


学習済みモデル（.h5）は TensorFlowTTS/examples/melgan の Pretrained Models and Audio samples からダウンロードした

In [20]:
# initialize melgan model
with open('examples/melgan/conf/melgan.v1.yaml') as f:
    melgan_config = yaml.load(f, Loader=yaml.Loader)
melgan_config = MelGANGeneratorConfig(**melgan_config["melgan_generator_params"])
melgan = TFMelGANGenerator(config=melgan_config, name='melgan_generator')
melgan._build()
melgan.load_weights("/content/drive/My Drive/Colab Notebooks/TensorFlowTTS/melgan.v1/checkpoints/generator-1500000.h5")

学習済みモデル（.h5）は TensorFlowTTS/examples/tacotron2 の Pretrained Models and Audio samples からダウンロードした

In [27]:
# initialize Tacotron2 model.
with open('examples/tacotron2/conf/tacotron2.v1.yaml') as f:
    config = yaml.load(f, Loader=yaml.Loader)
config = Tacotron2Config(**config["tacotron2_params"])
# 最新のTensorFlowTTS の TFTacotron2 には training 引数がなかったため削除
tacotron2 = TFTacotron2(config=config, name="tacotron2v2", enable_tflite_convertible=True)

# Newly added :
tacotron2.setup_window(win_front=6, win_back=6)
tacotron2.setup_maximum_iterations(3000)

tacotron2._build()
tacotron2.load_weights("/content/drive/My Drive/Colab Notebooks/TensorFlowTTS/tacotron2.v1/checkpoints/model-120000.h5")
tacotron2.summary()

Model: "tacotron2v2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder (TFTacotronEncoder)  multiple                  8218624   
_________________________________________________________________
decoder_cell (TFTacotronDeco multiple                  18246402  
_________________________________________________________________
post_net (TFTacotronPostnet) multiple                  5460480   
_________________________________________________________________
residual_projection (Dense)  multiple                  41040     
Total params: 31,966,546
Trainable params: 31,956,306
Non-trainable params: 10,240
_________________________________________________________________


### TFLite 形式へ変換

In [28]:
# Concrete Function
tacotron2_concrete_function = tacotron2.inference_tflite.get_concrete_function()

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [29]:
converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [tacotron2_concrete_function]
)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

変換したモデルのダウンロード。（自分のドライブへ）

In [31]:
# Save the TF Lite model.
with open('/content/drive/My Drive/Colab Notebooks/TensorFlowTTS/tacotron2.v1/tacotron2.tflite', 'wb') as f:
  f.write(tflite_model)

print('Model size is %f MBs.' % (len(tflite_model) / 1024 / 1024.0) )

Model size is 33.241852 MBs.
